__Ideas for how to make this work__


https://canvas.instructure.com/doc/api/peer_reviews.html

https://canvasapi.readthedocs.io/en/latest/course-ref.html

In [ ]:
%cd ~/Dropbox/CanvasHacks

#Plotting 
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

import pandas as pd
pd.options.display.max_rows = 999

from canvasapi import Canvas

from CanvasHacks import environment

# API access
# from CanvasHacks.RequestTools import *
# from CanvasHacks.UrlTools import *
# import CanvasHacks.DownloadProcessingTools as PT
from CanvasHacks.Api.UploadGradeTools import make_upload_button

# Filesystem
from CanvasHacks.Files.FileTools import makeDataFileIterator, create_folder
from CanvasHacks.TimeTools import getDateForMakingFileName

# Configuration
from CanvasHacks.Definitions.unit import Unit
from CanvasHacks.Definitions.discussion import DiscussionReview, DiscussionForum
from CanvasHacks.Configuration import InteractiveConfiguration

# Grading
# from CanvasHacks.GradingTools.nonempty import receives_credit, CreditForNonEmptyOLD #grade_credit_no_credit
from CanvasHacks.GradingHandlers.discussion import DiscussionForumGrader

# Models
from CanvasHacks.Models.student import Student
from CanvasHacks.Models.review_association import ReviewAssociation

# Notifications
# from CanvasHacks.PeerReviewed.Notifications import make_conversation_data, notify_student
# from CanvasHacks.PeerReviewed.Notifications import make_prompt_and_response, make_notice, send_message_to_reviewers
from CanvasHacks.Models.student import get_first_name

# Repos
from CanvasHacks.Repositories.DataManagement import DataStore
from CanvasHacks.Repositories.discussions import DiscussionRepository
from CanvasHacks.Repositories.students import StudentRepository
# from CanvasHacks.Repositories.codes import AccessCodeRepo
from CanvasHacks.Repositories.reviewer_associations import assign_reviewers, AssociationRepository

# Storage
from CanvasHacks.DAOs.sqlite_dao import SqliteDAO


# Widgets
from CanvasHacks.Widgets.ConsolidatedTextOutput import make_assignment_header, make_consolidated_text_fields
from CanvasHacks.Widgets.InputFields import make_course_ids_input, make_canvas_token_input, make_canvas_url_input, make_general_reset_button
from CanvasHacks.Widgets.AssignmentSelection import make_discussion_chooser
from CanvasHacks.Widgets.LiveSelection import make_test_selector
from CanvasHacks.Widgets.AssignmentSelection import make_unit_chooser, make_selection_button


import inspect
def look_inside(obj):
    print(inspect.getmembers(obj, lambda a:not(inspect.isroutine(a))))

from tests.factories.PeerReviewedFactories import discussion_entry_factory

LOC = '{}/Box Sync/TEACHING/Phil 305 Business ethics/Phil305 S20'.format(environment.ROOT)# placeholder for where the access codes are stored
SEMESTER_NAME = 'S20'

In [ ]:
make_test_selector()
make_unit_chooser()

In [ ]:
canvas = environment.CONFIG.canvas
course = environment.CONFIG.course
unit = environment.CONFIG.unit

# codeRepo = AccessCodeRepo(ACCESS_CODES_FP, environment.CONFIG.unit)

In [ ]:
q = course.get_quiz( rev.quiz_id )
q.access_code

In [ ]:
rev = unit.review
rev.__dict__

In [ ]:
from CanvasHacks.Widgets.DiscussionControls import discussion_run_button

In [ ]:
discussion_run_button()

# Handle sending posts to reviewers (as of CAN-44)

In [ ]:
from CanvasHacks.SkaaSteps.SendForumPostsToReviewer import SendForumPostsToReviewer

SEND = False
POST_THRESHOLD = 2
step = SendForumPostsToReviewer(environment.CONFIG.course, environment.CONFIG.unit, send=SEND, post_threshold=POST_THRESHOLD)

step._load_step()
# step.run() #download=False)


In [ ]:
len(step.work_repo.data)


In [ ]:
len(step.work_repo.filter_by_count(2))

In [ ]:
threshold = 2
students_to_keep = [sid for sid, cnt in step.work_repo.post_counts if cnt >= threshold]
[s for s in filter(lambda x: x['student_id'] in students_to_keep, step.work_repo.data)]
 # data.groupby('student_id').count()

In [ ]:
step.work_repo.data

In [ ]:
step.activity.is_discussion_type

In [ ]:
step._assign_step()

In [ ]:
step.associations

In [ ]:
assocs = step.dao.session.query(ReviewAssociation).filter(ReviewAssociation.activity_id == unit.discussion_review.id).all()
len(assocs)

In [ ]:
step.send=True
step.associations = assocs

In [ ]:
step._message_step()

In [ ]:
assocs

# Handle sending results to posters (as of CAN-44)

In [ ]:
from CanvasHacks.SkaaSteps.SendDiscussionReviewToPoster import SendDiscussionReviewToPoster
SEND = False
step = SendDiscussionReviewToPoster(environment.CONFIG.course, environment.CONFIG.unit, send=SEND)

step._load_step(rest_timeout=5)
# step.run(download=False)


In [ ]:
step._assign_step()

In [ ]:
step.send = True
step._message_step()

In [ ]:
step.dao.session.query(ReviewAssociation).filter(ReviewAssociation.activity_id == unit.discussion_review.id).all()


In [ ]:
# EVERYTHING BELOW NEEDS TO BE UPDATED TO WORK WITH CAN-44

# Teacher's review and grading

__Todo__:

    Update all methods to run on each repository in repos to allow for handling multiple classes/discussions

In [ ]:
make_discussion_chooser(environment.CONFIG.course)

In [ ]:
environment.CONFIG.unit.get_activity_by_id(797742)

In [ ]:
environment.CONFIG.discussions

In [ ]:
# If we haven't already gotten the discussion posts, do so for the selected discussions

try:
    if len(repos)>0:
        print("repos already populated")

except NameError:
    repos=[]
    for topic_id, name in environment.CONFIG.discussions:
        discussion_obj = environment.CONFIG.unit.get_discussion_by_topic_id(topic_id)
        print("Downloading posts for {}", name)
        discussionRepo = DiscussionRepository(course=environment.CONFIG.course, activity=discussion_obj)
        discussionRepo.download()
        repos.append(discussionRepo)

discussionRepo = repos[0]

studentRepo = StudentRepository(environment.CONFIG.course)
studentRepo.download()

## View posts

__Todo:__ 

    Only works right now for one discussion at a time


In [ ]:
discussionRepo.post_counts

### Display for teacher

__Todo:__ 
    
    Add buttons etc for modifying whether receives credit

In [ ]:
student_template = """
\n ============== {student_name}  {student_id} ============== \n
{posts}
\n ========================================================== \n
"""

# Collect each students' posts into one text block
# but don't display yet. This way the same results can either be displayed to teacher
# with identifiers or used to send to student reviewer
for_grading = []
for sid in discussionRepo.student_ids:
#     posts = [post_template.format(text=p) for p in discussionRepo.get_student_posts(sid)]
    
    posts = discussionRepo.get_student_posts(sid)
    posts = "".join(posts)
    j = { 'student_name' : studentRepo.get_student_name(sid),
    'student_id' : sid,
    'posts' : posts
    }
    for_grading.append(j)

len(for_grading)

In [ ]:
for d in for_grading:
    print(student_template.format(**d))

## Assign provisional grades

In [ ]:
num_posts_required = 1

grader = DiscussionForumGrader(discussionRepo, num_posts_required)
totals = grader.grade()
len(totals)

In [ ]:
# totals

## Review provisional grades 

In [ ]:
# Plot out how many students got what pct credit
tts = pd.Series([pct_credit for student_id, pct_credit in totals])
sns.countplot(tts)

In [ ]:
for s in results:
    make_assignment_header(s)
    make_consolidated_text_fields(s, studentRepo)

## Upload grades

In [ ]:
successes = 0; errors = 0
for student_id, pct_credit in totals:
    if student_id != environment.CONFIG.excluded_users:
        try:
            r = discussionRepo.upload_student_grade(student_id, pct_credit)
            successes += 1
        except AttributeError as e:
            print(student_id, pct_credit, e)
            errors += 1

print("Successfully uploaded grades for {} students".format(successes))
print("Error uploading grades for {} students".format(errors))

# Discussion reviews

In [ ]:
from CanvasHacks.Repositories.quizzes import QuizRepository, ReviewRepository
from CanvasHacks.QuizReportFileTools import sort_frames_by_age, get_newest_data

In [ ]:
make_unit_chooser()

In [ ]:
make_discussion_chooser(environment.CONFIG.course)

In [ ]:
create_folder(unit.discussion_review.folder_path)

## Load review results (from file)

In [ ]:
d = get_newest_data(environment.CONFIG.unit.discussion_review)

surveyRepo = ReviewRepository(environment.CONFIG.unit.discussion_review, environment.CONFIG.course)
surveyRepo.data = d
surveyRepo.set_question_columns(surveyRepo.data)
# fix_forgot_answers(surveyRepo)

In [ ]:
# Load things

if environment.CONFIG.is_test:
    # testing: in memory db
    dao = SqliteDAO()
    print("Connected to testing db")
else:
    db_filepath = "{}/{}-Unit-{}-discussion-review.db".format( environment.LOG_FOLDER, SEMESTER_NAME, environment.CONFIG.unit_number)
    # real: file db
    dao = SqliteDAO(db_filepath)
    dao.initialize_db_file()
    print("Connected to REAL db")

associationRepo = AssociationRepository(dao, environment.CONFIG.unit.discussion_review)

# If we haven't already gotten the discussion posts, do so for the selected discussions
try:
    if len(repos)>0:
        pass
except NameError:
    repos=[]
    for topic_id, name in environment.CONFIG.discussions:
        print("Downloading posts for {}", name)
        discussionRepo = DiscussionRepository(environment.CONFIG.course, topic_id)
        discussionRepo.download()
        repos.append(discussionRepo)
        print(topic_id)

discussionRepo = repos[0]

## Inspect distributions etc

In [ ]:
from CanvasHacks.VisualizationTools import rotate_x_labels

In [ ]:
rows = nrows=round(len(surveyRepo.multiple_choice_names)/2)
fig, axes = plt.subplots(ncols=2, nrows=rows, figsize=(12,15))
order = [l for l in environment.LIKERT_PLOT_ORDER if l != 'Forgot']

row=0; col=0
for c in surveyRepo.multiple_choice_names:
    title = c.split(':')[1][:65]
    g = sns.countplot(d[c], order=order, palette='plasma', ax=axes[row, col])
    g.set_xlabel('')
    axes[row, col].set_title(title)
    rotate_x_labels(axes[row, col])
    if col == 1:
        row += 1
        col = 0
    else:
        col += 1

fig.tight_layout()

## Check posts flagged as inconsistent with policies and not receiving full credit

In [ ]:
CONCERNING = ['Disagree', 'Strongly disagree']

In [ ]:
surveyRepo.make_question_selection_buttons()

In [ ]:
posts_for_review = []
data = surveyRepo.data.set_index('student_id')[surveyRepo.selected]

# rows in which the reviewer is expressing a concern about the performance
# of the student whom they reviewed
of_concern = data[data[surveyRepo.selected].isin(CONCERNING)].dropna(how='all')

# Combine the posts, ratings, and student info
for rid, row in of_concern.iterrows():
    aid = associationRepo.get_assessee(environment.CONFIG.unit.discussion_review, rid)
    p ={ 'author': aid,
     'reviewer': rid,
     'ratings' : row,
     'posts': discussionRepo.get_student_posts(aid)
    }
    posts_for_review.append(p)

print("Loaded posts and ratings for {} reviewees".format(len(posts_for_review)))

In [ ]:
# display
posts_for_review

# attic

In [ ]:
## Send feedback to person reviewed
posts_for_send = []
# This time we are getting everyone
data = surveyRepo.data.set_index('student_id')

# Combine the posts, ratings, and student info
for rid, row in data.iterrows():
    aid = associationRepo.get_assessee(environment.CONFIG.unit.discussion_review, rid)
    p ={ 'author': aid,
     'reviewer': rid,
     'ratings' : row,
     'posts': discussionRepo.get_student_posts(aid)
    }
    _for_send.append(p)

print("Loaded posts and ratings for {} reviewees".format(len(_for_send)))

In [ ]:
posts = []
problems = []
for sid, submission in discussionRepo.submissions.items():
    for entry in submission.discussion_entries:
        try:
            posts.append( (entry['user_id'], entry['user_name'], entry['message']) )
        except:
            problems.append(entry)
len(posts)

In [ ]:


def get_discussion_entries(course, topic_id):
    """Returns a list of discussion objects. 
        Objects look something like:
        {'id': 2132485, 'user_id': 169155, 
        'parent_id': None, 'created_at': '2020-01-16T23:01:53Z', 
        'updated_at': '2020-01-16T23:01:53Z', 'rating_count': None,
        'rating_sum': None, 'user_name': 'Test Student', 
        'message': '<p>got em</p>', 'user': {'id': 169155, 
        'display_name': 'Test Student',
        'avatar_image_url': 'https://canvas.csun.edu/images/messages/avatar-50.png',
        'html_url': 'https://canvas.csun.edu/courses/85210/users/169155',
        'pronouns': None, 'fake_student': True}, 
        'read_state': 'unread', 'forced_read_state': False, 
        'discussion_id': 737847, 'course_id': 85210}
    """
    discussion = course.get_discussion_topic(topic_id)
    # result is lazy loaded, so iterate through it
    return [e for e in discussion.get_topic_entries()]
 

def parse_discussion_results(results):
    """Returns a list of tuples of all dicussion posts for the topic
    Format:
        [ ( user_id, text )]
        """
    return [( e.user_id, e.message ) for e in results]
    
#     print(e.attributes)
#     look_inside(e.attributes)

In [ ]:
sub = discussionRepo.submissions[22946]
sub.discussion_entries

def parse_posts_from_submission(submission):
    for entry in submisson.discussion_entries:
        self.posts.append( (entry.user_id, entry.user_name, entry.message) )

    return (subm)

In [ ]:

def assign_grades(discussion_repo, num_posts_required):
    credit_per_post = 100 / num_posts_required

    grades = [ 
    # ( sid, percent credit)
    ]

    for p in discussion_repo.data:
    #     print(post, grade_credit_no_credit(post))
#         credit_per_post
        pct_credit = credit_per_post if grade_credit_no_credit(p['text']) else 0
        grades.append( (p['student_id'], pct_credit) )

    # sum them up and put in list for upload
    totals = []
    sids = list(set([s[0] for s in grades]))
    for sid in sids:
        t = sum([s[1] for s in grades])
        t = 100 if t > 100 else t
        totals.append( ( sid,  t)) 
    return totals

# totals

In [ ]:
# Provide to reviewer (old)
if environment.CONFIG.is_test:
    # testing: in memory db
    dao = SqliteDAO()
    print("Connected to testing db")
else:
    db_filepath = "{}/{}-Unit-{}-discussion-review.db".format( environment.LOG_FOLDER, SEMESTER_NAME, environment.CONFIG.unit_number)
    # real: file db
    dao = SqliteDAO(db_filepath)
    dao.initialize_db_file()
    print("Connected to REAL db")

associationRepo = AssociationRepository(dao, environment.CONFIG.unit.discussion_review)

# all students who submitted the assignment
# submitters = [169908, 169955, 169957]
submitters = discussionRepo.student_ids

# Assign reviewers to each submitter and store in db
associationRepo.assign_reviewers( submitters)

if not environment.CONFIG.is_test:
    # Make audit file
    review_audit = []
    for rev in associationRepo.get_associations(unit.discussion_review):
        print(rev.assessor_id, rev.assessee_id)
        assessor = studentRepo.get_student(rev.assessor_id)
        assessee = studentRepo.get_student(rev.assessee_id)
        print(assessor)

        review_audit.append({
            'activity' : unit.discussion_review.name,
            'assessor_name' : assessor.short_name,
            'assessor_sis_id': assessor.sis_user_id,
            'assessor_canvas_id': assessor.id,
            'assessee_name' : assessee.short_name,
            'assessee_sis_id': assessee.sis_user_id,
            'assessee_canvas_id': assessee.id,
            'timestamp': datetime.datetime.now().isoformat()
        })

    review_audit = pd.DataFrame(review_audit)

    fp = "{}/{}-Unit{}-peer review assignments.csv".format(environment.LOG_FOLDER, getDateForMakingFileName(), environment.CONFIG.unit )
    review_audit.to_csv(fp)
    print("Audit file saved to {}".format(fp))
    # review_audit

SEND = False
# SEND = True

# make sure access code set on activity before running until CAN-32 is done!

send_message_to_reviewers(associationRepo, studentRepo, discussionRepo, unit.discussion_review, SEND)

# Rubrics
 GET /api/v1/courses/:course_id/rubrics/:id 
 
 __this is all wasted time. Rubrics can only be used by instructor not given to students for peer review__

In [ ]:
assign_id = 568253
assign = course.get_assignment(assign_id)

In [ ]:
# will need submissions to look up graded student info
subs_repo = SubmissionRepository(assign)

In [ ]:
reviews = [r for r in assign.get_peer_reviews()]
reviews

In [ ]:
subs = [ s for s in assign.get_submissions()]
subs[0]

In [ ]:
rubrics = [r for r in course.get_rubrics()]
rubrics

In [ ]:
[ 'assessments', 'graded_assessments', 
                                       'peer_assessments', 'associations', 
                                       'assignment_associations', 
                                       'course_associations', 'account_associations'])

In [ ]:
rubric = course.get_rubric(14141, style='full', include=[  'graded_assessments'])
rubric

In [ ]:
assign_id = 
artifact_id = 22191947

In [ ]:
rubric.rubric_association

rubric.assessments

Actual graded rubrics

{'id': 239043,
  'rubric_id': 14141,
  'rubric_association_id': 28259,
  
  Total score
  'score': 10.0,
  'artifact_id': 22191947,
  'artifact_type': 'Submission',
  'assessment_type': 'grading',
  
  Person grading
  'assessor_id': 6417,
  'artifact_attempt': 2,
  
  'data': [{'id': None,
    'points': 2.0,
    'criterion_id': '_9091',
    'learning_outcome_id': None,
    'description': 'No details',
    'comments_enabled': True,
    'comments': ''},

What we need for peer review graded discussion posts (CAN-16)

- grading student

- graded student

- all scores

- all comments

In [ ]:

def get_graded_student_id(rubric_assessment, submissions):
    submission_id = rubric_assessment['artifact_id']
    for s in submissions:
        if s.id == submission_id:
            return s.user_id

In [ ]:
get_graded_student_id(rubric.assessments[0], subs)

In [ ]:
assign_id = rubric.assessments[0]['rubric_association']['association_id']
print(assign_id)
assign = course.get_assignment(assign_id)
# assign.get_submission(22191947)
for s in assign.get_submissions():
    print(s)
    
    
assign.get_submission(169155)
    

In [ ]:
rubric.assessments


In [ ]:
# # TEST = True
# TEST = False
# SEMESTER_NAME = 'S20'

# if TEST:
#     environment.CONFIG.set_test()
# # environment.CONFIG.set_live()
# ACCESS_CODES_FP = "{}/{}-assignment-access-codes.xlsx".format(LOC, SEMESTER_NAME)
# COURSE_ID = environment.CONFIG.course_ids[0]
# COURSE_ID

PROF_ID = 6417

# Test data
test_course_id = 85210

main_quiz = 165098
review_quiz = 165820
meta_review = 165821

url = 'https://canvas.csun.edu/api/v1/courses/85210/assignments/{}/submissions'

# last semester data
prev_course_id = 62657
prev_quiz = 151633

COURSE_ID = test_course_id
# COURSE_ID = prev_course_id
# QUIZ_ID = prev_quiz

TEST_STUDENT_ID = 168439



topic_id = 737847
graded_topic_id = 768729

In [ ]:
def make_question_selection_buttons(repo):
    """Given a repository containing a dataframe and a 
    list of names in question_names, this will allow to select
    which questions are used for things"""
    buttons = []
    for q in repo.question_names:
        b = make_selection_button(q, q, repo.get_selections, repo.select, repo.deselect, '100%' )
        buttons.append(b)
        

In [ ]:
download_url = parse_download_url(response)

response_content_for_testing = b'{"id":91464,"report_type":"student_analysis","readable_type":"Student Analysis","includes_all_versions":false,"includes_sis_ids":true,"generatable":true,"anonymous":false,"url":"https://canvas.csun.edu/api/v1/courses/85210/quizzes/165820/reports/91464","progress_url":"https://canvas.csun.edu/api/v1/progress/335965","created_at":"2019-12-27T01:45:29Z","updated_at":"2019-12-27T01:55:54Z","file":{"id":7868563,"uuid":"VZNHEVLsfBZt2IJYCvkBdmM2x5LfqIBVjXoRmDVo","folder_id":null,"display_name":"Quiz1-review Survey Student Analysis Report.csv","filename":"quiz_student_analysis_report.csv","upload_status":"success","content-type":"text/csv","url":"https://canvas.csun.edu/files/7868563/download?download_frd=1\\u0026verifier=VZNHEVLsfBZt2IJYCvkBdmM2x5LfqIBVjXoRmDVo","size":132,"created_at":"2019-12-27T01:55:54Z","updated_at":"2019-12-27T01:55:54Z","unlock_at":null,"locked":false,"hidden":false,"lock_at":null,"hidden_for_user":false,"thumbnail_url":null,"modified_at":"2019-12-27T01:55:54Z","mime_class":"file","media_entry_id":null,"locked_for_user":false},"quiz_id":165820}'

In [ ]:
data = {
#     'quiz_report': [{
#         'report_type': 'student_analysis',
#     }],
    "include": ["progress", "file"]
}

# This report url works properly!!!!!!!!!!!!!!!!
report_url = 'https://canvas.csun.edu/api/v1/courses/85210/quizzes/165820/reports/91464'
# Make the request 
# We can't use the usual function since we don't want response.json()
head = { 'Authorization': 'Bearer {}'.format( environment.CONFIG.canvas_token ) }
response = requests.get( report_url, headers=make_request_header(), json=data )

print('response was ', response.status_code)


In [ ]:
# report_url = 'https://canvas.csun.edu/api/v1/courses/85210/quizzes/165820/reports/91464'

response.content